In [19]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right

In [69]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning

TRESH_DIST = 3 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436 #150 speed
YAW_TO_MMS = 0.06086 # -100 100

def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        
        # we need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_obstacle.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_path.jpg', frame_init) #POUR TUNING PAR ELIOTT

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame.
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
        x_est_t_minus_1, hidden_cam = computer_vision.vision(frame, px_to_mm)
        # x_est_t_minus_1 = np.array([0.0,0.0,0.0])
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[0.1,  0,   0],
                                [  0,0.1,   0],
                                [  0,  0, 0.1]])
    
    
        #update le dt car sinon kalman marche pas
        dt = 0.1 # EN SECONDES POUR KALMAN!!!!! same as PI_CLOCK of pid defined in filters?
        sum_error = 0 
        alt_error_pid = 0
        
        time.sleep(0.1)
        
        while len(goal_list)!=0:
            
            print("goal is: ", goal_list[0])
            
            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
#             cv2.imshow('running frame', frame)

                
            ##### APPELER get_vision_position##########
            obs_vector_z_t, hidden_cam = computer_vision.vision(frame, px_to_mm)
            
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, is_from_camera = True)
            
            
#             print(f'Timestep measurement={obs_vector_z_t}')
    
            # Run the Extended Kalman Filter and store the 
            # near-optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.ekf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Time interval
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, is_from_camera = False)
            
            frame = cv2.resize(frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            #check if obstacle in coming
            
            #if no
            ######APPELER PID##########
            print()
            sum_error, alt_error_pid, v_l,v_d = filters.pid(optimal_state_estimate_t,goal_list[0],sum_error, alt_error_pid,dt)
            v_l = 50
            v_d = 50
            v = (v_l + v_d)*SPEED_TO_MMS/2
            yaw = (v_d - v_l)*YAW_TO_MMS
            u_t_minus_1 = [v, yaw]; # delta_v returned by PID
#             u_t_minus_1 = 0;
            set_speed(int(v_l), int(v_d))

    
    
            #if yes, obstacle avoidance 
            
            actual = time.time()
            diff = actual-previous
            print("elapsed time is: ", diff)
            if diff<dt:
                time.sleep(dt-diff)
            
    finally: 
        cv2.destroyAllWindows()



In [26]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [27]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [10]:
import tdmclient.notebook
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [70]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)

<module 'computer_vision' from 'C:\\Users\\papad\\OneDrive\\Documents\\MA1\\Basic_of_mobile_robotics\\project\\Mobile_robotics\\computer_vision.py'>

In [71]:
main()

take the rigth image


100%|██████████| 1/1 [00:00<00:00, 1014.83it/s]


goal is:  [94.126 69.281]
Robot not seen by camera
posa : [197, 145] <class 'list'>

pos robot is:  [94.136 69.291 33.182]
elapsed time is:  0.07211852073669434
goal is:  [193.508 269.956]
posa : [196, 145] <class 'list'>
posa : [196, 146] <class 'list'>

pos robot is:  [93.69  70.056 32.9  ]
elapsed time is:  0.08317780494689941
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [196, 150] <class 'list'>

pos robot is:  [93.85  71.778 32.903]
elapsed time is:  0.0884406566619873
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [196, 153] <class 'list'>

pos robot is:  [94.004 73.5   32.906]
elapsed time is:  0.04879117012023926
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [197, 157] <class 'list'>

pos robot is:  [94.154 75.222 32.909]
elapsed time is:  0.07067108154296875
goal is:  [193.508 269.956]
posa : [201, 148] <class 'list'>
posa : [199, 153] <class 'list'>

pos robot is:  [95.306 73.141 33.049]
elapsed time is:  0.05347847938537598
goal is: 

posa : [281, 208] <class 'list'>
posa : [282, 206] <class 'list'>

pos robot is:  [135.214  98.848  37.645]
elapsed time is:  0.06147003173828125
goal is:  [193.508 269.956]
posa : [284, 209] <class 'list'>
posa : [284, 208] <class 'list'>

pos robot is:  [136.127  99.398  37.813]
elapsed time is:  0.04997539520263672
goal is:  [193.508 269.956]
posa : [285, 211] <class 'list'>
posa : [286, 209] <class 'list'>

pos robot is:  [136.768 100.309  37.537]
elapsed time is:  0.04258275032043457
goal is:  [193.508 269.956]
posa : [287, 212] <class 'list'>
posa : [287, 210] <class 'list'>

pos robot is:  [137.599 100.772  37.866]
elapsed time is:  0.047615766525268555
goal is:  [193.508 269.956]
posa : [289, 214] <class 'list'>
posa : [289, 212] <class 'list'>

pos robot is:  [138.506 101.754  37.558]
elapsed time is:  0.0702366828918457
goal is:  [193.508 269.956]
posa : [290, 215] <class 'list'>
posa : [291, 213] <class 'list'>

pos robot is:  [139.446 102.223  37.874]
elapsed time is:  0.05

posa : [375, 282] <class 'list'>
posa : [375, 281] <class 'list'>

pos robot is:  [179.53  134.594  37.919]
elapsed time is:  0.06266498565673828
goal is:  [193.508 269.956]
posa : [377, 283] <class 'list'>
posa : [377, 282] <class 'list'>

pos robot is:  [180.502 135.083  37.868]
elapsed time is:  0.05161547660827637
goal is:  [193.508 269.956]
posa : [379, 285] <class 'list'>
posa : [379, 284] <class 'list'>

pos robot is:  [181.47  135.827  37.849]
elapsed time is:  0.07260704040527344
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [383, 284] <class 'list'>

pos robot is:  [183.179 136.093  37.852]
elapsed time is:  0.06371688842773438
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [386, 285] <class 'list'>

pos robot is:  [184.887 136.364  37.855]
elapsed time is:  0.06668877601623535
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [390, 285] <class 'list'>

pos robot is:  [186.594 136.641  37.858]
elapsed time is:  0.054772138595581055
goal is

...

Robot not seen by camera
posa : [175, 223] <class 'list'>

pos robot is:  [ 84.014 106.863  21.409]
elapsed time is:  0.06557989120483398
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [172, 225] <class 'list'>

pos robot is:  [ 82.589 107.818  21.412]
elapsed time is:  0.06575989723205566
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [169, 227] <class 'list'>

pos robot is:  [ 81.161 108.768  21.415]
elapsed time is:  0.05694317817687988
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [166, 229] <class 'list'>

pos robot is:  [ 79.731 109.714  21.418]
elapsed time is:  0.04459571838378906
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [163, 231] <class 'list'>

pos robot is:  [ 78.297 110.656  21.421]
elapsed time is:  0.06687474250793457
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [160, 233] <class 'list'>

pos robot is:  [ 76.861 111.593  21.424]
elapsed time is:  0.07288765907287598
goal is:  [193.508 269.956]
Robo

Robot not seen by camera
posa : [265, 249] <class 'list'>

pos robot is:  [126.661 119.342  21.909]
elapsed time is:  0.04722118377685547
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [261, 250] <class 'list'>

pos robot is:  [124.958 119.494  21.912]
elapsed time is:  0.0669853687286377
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [257, 250] <class 'list'>

pos robot is:  [123.256 119.64   21.915]
elapsed time is:  0.05726885795593262
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [254, 250] <class 'list'>

pos robot is:  [121.553 119.781  21.918]
elapsed time is:  0.047333717346191406
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [250, 250] <class 'list'>

pos robot is:  [119.85  119.917  21.921]
elapsed time is:  0.05630850791931152
goal is:  [193.508 269.956]
Robot not seen by camera
posa : [247, 251] <class 'list'>

pos robot is:  [118.146 120.048  21.924]
elapsed time is:  0.07450413703918457
goal is:  [193.508 269.956]
Robo

posa : [483, 306] <class 'list'>

pos robot is:  [231.229 146.404  22.027]
elapsed time is:  0.05939960479736328
goal is:  [193.508 269.956]
posa : [490, 309] <class 'list'>
posa : [486, 307] <class 'list'>

pos robot is:  [232.321 147.104  21.598]
elapsed time is:  0.051248788833618164
goal is:  [193.508 269.956]
posa : [492, 310] <class 'list'>
posa : [488, 309] <class 'list'>

pos robot is:  [233.379 147.943  21.434]
elapsed time is:  0.05715537071228027
goal is:  [193.508 269.956]
posa : [496, 311] <class 'list'>
posa : [491, 311] <class 'list'>

pos robot is:  [235.013 148.654  20.688]
elapsed time is:  0.07302117347717285
goal is:  [193.508 269.956]
posa : [498, 311] <class 'list'>
posa : [495, 312] <class 'list'>

pos robot is:  [236.611 149.211  21.529]
elapsed time is:  0.06489872932434082
goal is:  [193.508 269.956]
posa : [500, 313] <class 'list'>
posa : [496, 313] <class 'list'>

pos robot is:  [237.398 149.674  21.408]
elapsed time is:  0.05289506912231445
goal is:  [193.5

posa : [612, 357] <class 'list'>
posa : [608, 355] <class 'list'>

pos robot is:  [290.915 169.781  21.574]
elapsed time is:  0.05769920349121094
goal is:  [193.508 269.956]
posa : [614, 358] <class 'list'>
posa : [610, 357] <class 'list'>

pos robot is:  [291.792 170.792  21.085]
elapsed time is:  0.04649543762207031
goal is:  [193.508 269.956]
posa : [618, 359] <class 'list'>
posa : [614, 359] <class 'list'>

pos robot is:  [293.503 171.725  20.898]
elapsed time is:  0.07143044471740723
goal is:  [193.508 269.956]
posa : [620, 360] <class 'list'>
posa : [617, 360] <class 'list'>

pos robot is:  [294.85  172.443  22.482]
elapsed time is:  0.06162714958190918
goal is:  [193.508 269.956]
posa : [623, 361] <class 'list'>
posa : [619, 360] <class 'list'>

pos robot is:  [295.973 172.12   21.432]
elapsed time is:  0.07173490524291992
goal is:  [193.508 269.956]
posa : [625, 361] <class 'list'>
posa : [621, 361] <class 'list'>

pos robot is:  [297.016 172.655  21.464]
elapsed time is:  0.05

KeyboardInterrupt: 

In [8]:
cap.release()

In [72]:
set_speed(0, 0)